<a href="https://colab.research.google.com/github/Maxny87/theory_of_comp/blob/main/theory_of_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import itertools

dfa_contains_a = {
    "states": [0, 1],
    "alphabet": ["a", "b"],
    "transitions": {
        0: {"a": 1, "b": 0},   # if we see a go to accepting state
        1: {"a": 1, "b": 1},   # once we saw a stay accepting
    },
    "start_state": 0,
    "accept_states": [1]
}


dfa_empty_string_only = {
    "states": [0, 1],
    "alphabet": ["a", "b"],
    "transitions": {
        # Any symbol goes to state 1 which is a dead state
        0: {"a": 1, "b": 1},
        1: {"a": 1, "b": 1},
    },
    "start_state": 0,
    "accept_states": [0]
}

dfa_missing_transition = {
    "states": [0, 1],
    "alphabet": ["a", "b"],
    "transitions": {
        0: {"a": 1, "b": 0},
        1: {"a": 1}  # b is missing for state 1
    },
    "start_state": 0,
    "accept_states": [1]
}



In [ ]:
# ==== Validate DFAs ====

def validate_dfa(dfa):

    required = ["states", "alphabet", "transitions", "start_state", "accept_states"]
    for prop in required:
        if prop not in dfa:
            return f"Invalid DFA: Missing property '{prop}'." # need all of them in dfa

    states = dfa["states"]
    alphabet = dfa["alphabet"]
    transitions = dfa["transitions"]
    start = dfa["start_state"]
    accepts = dfa["accept_states"]

    if len(states) != len(set(states)):
        return "Invalid DFA: states must be unique."

    if len(alphabet) != len(set(alphabet)):
        return "Invalid DFA: alphabet must be unique."

    if not isinstance(transitions, dict):
        return "Invalid DFA: transitions must be a dict."

    # check transitions for each state
    for s in states:
        if s not in transitions:
            return f"Invalid DFA: Missing transitions for state '{s}'."
        for sym in alphabet:
            if sym not in transitions[s]:
                return f"Invalid DFA: Missing transition for state {s} on symbol {sym}." # every state needs transition for evert letter in alphabet
            if transitions[s][sym] not in states:
                return f"Invalid DFA: Transition leads to invalid state."

    if start not in states:
        return "Invalid DFA: start state invalid."

    if not isinstance(accepts, list):
        return "Invalid DFA: accept_states must be a list."

    for a in accepts:
        if a not in states:
            return f"Invalid DFA: accept state {a} invalid."

    return "DFA is valid."

def validate_dfa_binary(dfa):
    return 1 if validate_dfa(dfa) == "DFA is valid." else 0




# ==== Determine if DFA recognizes emptry string ====

def dfa_accepts_empty_language(dfa):
    """
    If DFA has any reachable accept states, then it accepts more than just the empty language.

    If no reachable accept states, then it accepts the empty language.
    """


    if validate_dfa(dfa) != "DFA is valid.":
        raise ValueError("Invalid DFA")

    transitions = dfa["transitions"]
    start = dfa["start_state"]
    accepts = set(dfa["accept_states"])
    alphabet = dfa["alphabet"]

    # bfs
    visited = set()
    queue = [start]

    while queue:
        state = queue.pop(0)
        if state in visited:
            continue
        visited.add(state)

        if state in accepts:
            return False  # gets to accept state - not empty

        for sym in alphabet:
            next_state = transitions[state][sym]
            if next_state not in visited:
                queue.append(next_state)
    return True



# ==== Enumerate all DFAs ====

def simulate_dfa(dfa, string):
    if validate_dfa(dfa) != "DFA is valid.":
        raise ValueError("Invalid DFA")

    current = dfa["start_state"]
    transitions = dfa["transitions"]

    for symbol in string:
        if symbol not in dfa["alphabet"]:
            raise ValueError(f"Symbol {symbol} not in alphabet.")
        current = transitions[current][symbol] # next state is the result of being in state s and input symbol

    return current in dfa["accept_states"] # whether in accept state or not

def enumerate_dfas(alphabet=["a", "b"]):
    count = 0
    n = 1  # num of states in dfa

    while True:
        # we need
        # every possible transition table
        # every possible start state
        # every possible accept-state subset

        states = list(range(n))

        # create list of (state, symbol) pairs: list comprehension (double for loop)
        transition_keys = [(s, sym) for s in states for sym in alphabet] # create list of (state, symbol) pairs

        # Cartesian product of possible targets
        # Example: for n=2 and alphabet={a,b}, we need 4 choices, each in [0,1]
        for targets in itertools.product(states, repeat=len(transition_keys)):



            # Build transition dict
            transitions = {s: {} for s in states}
            for ((state, sym), target) in zip(transition_keys, targets):
                transitions[state][sym] = target

            # Choose start state
            for start in states:
                # Choose ALL possible accept sets (power set)
                for r in range(len(states) + 1):
                    for accept_subset in itertools.combinations(states, r):

                        # Construct DFA dictionary
                        dfa = {
                            "states": states,
                            "alphabet": alphabet,
                            "transitions": transitions,
                            "start_state": start,
                            "accept_states": list(accept_subset)
                        }

                        yield dfa  # produce DFA

        n += 1  # increase number of states




# ==== DFA to TM ====

def dfa_to_tm(dfa):
    states = dfa["states"]
    alphabet = dfa["alphabet"]
    transitions = dfa["transitions"]
    start = dfa["start_state"]
    accepts = dfa["accept_states"]

    tm = {
        "states": states + ["HALT_ACCEPT", "HALT_REJECT"],
        "input_alphabet": alphabet,
        "tape_alphabet": alphabet + ["_"],
        "start_state": start,
        "accept_states": ["HALT_ACCEPT"],
        "reject_states": ["HALT_REJECT"],
        "transitions": {}
    }

    # convert DFA transitions to TM transitions (write same symbol, move right)
    for q in states:
        for a in alphabet:
            tm["transitions"][(q, a)] = (transitions[q][a], a, "R")

    # When reading blank, halt
    for q in states:
        if q in accepts:
            tm["transitions"][(q, "_")] = ("HALT_ACCEPT", "_", "R")
        else:
            tm["transitions"][(q, "_")] = ("HALT_REJECT", "_", "R")

    return tm

def simulate_tm(tm, input_string, verbose=True):
    # initialize tape
    tape = list(input_string) + ["_"]
    head = 0
    state = tm["start_state"]

    step = 0

    while True:
        symbol = tape[head]

        if (state, symbol) not in tm["transitions"]:
            # no transition defined → reject
            state = "HALT_REJECT"

        next_state, write, direction = tm["transitions"][(state, symbol)]
        tape[head] = write

        # verbose output
        if verbose:
            print(f"Step {step}:")
            print("State:", state)
            print("Tape: ", "".join(tape))
            print("       " + " " * head + "^")
            print()

        # move
        if next_state == "HALT_ACCEPT":
            print("TM HALTS ACCEPT")
            return True
        if next_state == "HALT_REJECT":
            print("TM HALTS REJECT")
            return False

        state = next_state
        if direction == "R":
            head += 1
            if head == len(tape):
                tape.append("_")

        step += 1